In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import time

from model import Generator, Discriminator
from config import irgan_config
from data_utils_1 import RecDataset, DataProvider
from evaluation.rec_evaluator import RecEvaluator

In [2]:
epochs = irgan_config.epochs
batch_size = irgan_config.batch_size
epochs_d = irgan_config.epochs_d
epochs_g = irgan_config.epochs_g
emb_dim = irgan_config.emb_dim
eta_G = irgan_config.eta_G
eta_D = irgan_config.eta_D
device = irgan_config.device
weight_decay_g = irgan_config.weight_decay_g
weight_decay_d = irgan_config.weight_decay_d

In [3]:
# Hyper-parameters and datset-specific parameters
rec_dataset = RecDataset("./data/ml-1m/")
all_users = rec_dataset.get_users()
all_items = rec_dataset.get_items()
num_users = rec_dataset.get_num_users()
num_items = rec_dataset.get_num_items()
bought_mask = rec_dataset.get_bought_mask().to(device)
eval_dict = rec_dataset.get_interaction_records("test")
train_ui = rec_dataset.get_user_item_pairs()

In [5]:
G = Generator(num_users, num_items, emb_dim, bought_mask)
D = Discriminator(num_users, num_items, emb_dim)
G = G.to(device)
D = D.to(device)

loss_D = nn.BCELoss()
optimizer_G = torch.optim.SGD(G.parameters(), momentum = 0.9, lr = eta_G)
optimizer_D = torch.optim.SGD(D.parameters(), momentum = 0.9, lr = eta_D, weight_decay = weight_decay_d)

In [6]:
for epoch in range(epochs):
    print(
            "[Epoch %d/%d]"
            % (epoch+1, epochs)
        )  
    time_epoch_start = time.time()
    
    # ---------------------
    #  Train Discriminator
    # ---------------------   
    D.train()
    G.eval()
    for epoch_d in range(epochs_d):
        time_d_start = time.time()
        if(epoch_d % 5 == 0):
            train_set_d = dp.prepare_data_for_discriminator(G, train_ui, batch_size = batch_size)
        loss_d_epoch = 0
        for users, items, labels in train_set_d:
            dis_score = D(users,items)
            loss_d = loss_D(dis_score, labels)
            loss_d_epoch += loss_d.item()
            optimizer_D.zero_grad()
            loss_d.backward()
            optimizer_D.step()
            
        time_d_end = time.time()
        loss_d_epoch /= len(train_set_d)
        print(
            "\t[Discriminator][Epochs %d/%d] [D epoch loss: %6.5f] [Time:%6.5f] "
            % (epoch_d+1, epochs_d, loss_d_epoch, time_d_end - time_d_start)
        )
       
        with torch.no_grad():

            torch.save(D.state_dict(),"./pretrained_models/ml-1m/pretrained_model_discriminator.pkl")
        
    # --------------------- 
    #  Train Generator
    #
    # For generator
    #   Generate K user-item pairs
    #   Leveraging Policy Gradient to update parameters of generator
    #
    # --------------------- 
    D.eval()
    G.train()
    for epoch_g in range(epochs_g):
        time_g_start = time.time()
        train_set_g = dp.prepare_data_for_generator(all_users, batch_size)
        loss_g_epoch = 0
        for fake_users, in train_set_g:
            fake_items, fake_probs, fake_p_n = G.sample_items_for_users(fake_users, k = 256,\
                                                                        temperature=1, lambda_bought=0.2)
            fake_users = fake_users.view(-1,1).expand_as(fake_items).contiguous()
            fake_users = fake_users.view(-1)
            fake_items = fake_items.view(-1)
            fake_probs = fake_probs.view(-1)
            fake_p_n = fake_p_n.view(-1)
            
            log_fake_probs = torch.log(fake_probs.clamp(1e-8))
            fake_probs = fake_probs.detach()
            fake_p_n = fake_p_n.detach()
            
            reward = (2 * D(fake_users, fake_items) - 1).detach()*(fake_probs/fake_p_n)
            loss_g = -torch.mean(log_fake_probs*reward)
            loss_g_epoch += loss_g
            
            optimizer_G.zero_grad()  
            loss_g.backward() 
            optimizer_G.step()
        time_g_end = time.time()
        print(
            "\t[Generator][Epochs %d/%d] [G epoch loss: %6.5f] [Time:%6.5f]"
            % (epoch_g + 1, epochs_g, loss_g_epoch, time_g_end - time_g_start )
        ) 
     
        with torch.no_grad():

            torch.save(G.state_dict(),"./pretrained_models/ml-1m/pretrained_model_generator.pkl")
    time_epoch_end = time.time()


[Epoch 1/100]
	[Discriminator][Epochs 1/100] [D epoch loss: 0.68940] [Time:34.11471] 


KeyboardInterrupt: 